In [ ]:
#| default_exp core

# core

> Fill in a module description here

In [ ]:
from aimagic import *
import re, types, xml.etree.ElementTree as ET
from functools import partial

In [ ]:
def strip_ansi_codes(text):
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

In [ ]:
from toolslm import xml

In [ ]:
m = create_magic(models[0])

In [ ]:
def _attrmap(o):
    o = o.lstrip('_').replace('_','-')
    return dict(cls='class', klass='class', fr='for').get(o, o)

def xt(tag:str, *c, **kw):
    if len(c)==1 and isinstance(c[0], types.GeneratorType): c = tuple(c[0])
    elif len(c)==1 and isinstance(c[0],str): c = c[0]
    kw = {_attrmap(k):str(v) for k,v in kw.items()}
    return [tag.lower(),c,kw]

g = globals()
tags = '''html head title meta link style body pre code div span p h1 h2 h3 h4 h5 h6 strong em b i u s
strike sub sup hr br img a link nav ul ol li dl dt dd table thead tbody tfoot tr th td caption
col colgroup form input textarea button select option label fieldset legend details summary main
header footer section article aside figure figcaption mark small iframe object embed param video
audio source canvas svg math script noscript template slot'''.split()
for o in tags: g[o.capitalize()] = partial(xt, o)

def to_xml(node:tuple, hl=False):
    "Convert `node` to an XML string."
    def mk_el(tag, cs, attrs):
        el = ET.Element(tag, attrib=attrs)
        if isinstance(cs, tuple): el.extend([mk_el(*o) for o in cs])
        elif cs is not None: el.text = str(cs)
        return el

    root = mk_el(*node)
    ET.indent(root, space='  ' if hl else '')
    res = ET.tostring(root, encoding='unicode', short_empty_elements=False)
    return hl_md(res) if hl else res

In [ ]:
# to_xml(Body(H1('hi'), P('foo')), True)

In [ ]:
%ai reset

In [ ]:
from execnb import nbio
import json, os
from fastcore.utils import *

In [ ]:
%%aio 0
Tell me a detailed step by step plan to open a Jupyter Notebook with execnb, and create a rendered HTML version of it, without using nbformat or anything similar -- instead doing it by hand from scratch.

-  Load the notebook JSON data from a file: `nb_data = json.load(open('notebook.ipynb'))`
-  Initialize an empty list to store the HTML output: `html_output = []`
-  Create a basic HTML page, including a javascript import of a markdown rendering library
-  Iterate through each cell in the notebook data:
    -  If the cell is a code cell, convert to an HTML representation (we will assume the outputs are already saved in the notebook)
    -  If the cell is a markdown cell, output it in a format the js md library can render
-  Combine the HTML output from each cell into a single string
-  Write the HTML string to a file: `with open('notebook.html', 'w') as f: f.write(html_output)`

In [ ]:
%%aip 0
Do the 1st 2 steps.

In [ ]:
fname = Path.home()/'git/nbdev/tests/minimal.ipynb'
nb = nbio.read_nb(fname)

In [ ]:
nb

```json
{ 'cells': [ { 'cell_type': 'markdown',
               'idx_': 0,
               'metadata': {},
               'source': '## A minimal notebook'},
             { 'cell_type': 'code',
               'execution_count': None,
               'idx_': 1,
               'metadata': {},
               'outputs': [ { 'data': {'text/plain': ['2']},
                              'execution_count': None,
                              'metadata': {},
                              'output_type': 'execute_result'}],
               'source': '# Do some arithmetic\n1+1'}],
  'metadata': { 'kernelspec': { 'display_name': 'Python 3 (ipykernel)',
                                'language': 'python',
                                'name': 'python3'}},
  'nbformat': 4,
  'nbformat_minor': 4,
  'path_': '/Users/jhoward/git/nbdev/tests/minimal.ipynb'}
```

In [ ]:
%%ai 0
Great. Show me the script to auto-render cells with a certain class using `marked`. If my app creates cells of that class, I want them auto-formatted.

Place a script block with this at the end of your page:

```js
  const contentDiv = document.getElementById('content');
  const renderMarkdown = (block) => { block.innerHTML = marked.parse(block.innerHTML) };
  const observer = new MutationObserver((mutations) => {
    mutations.forEach((mutation) => {
      if (mutation.type === 'childList') {
        mutation.addedNodes.forEach((node) => {
          if (node.nodeType === Node.ELEMENT_NODE && node.classList.contains('markdown'))
          renderMarkdown(node);
        });
      }
    });
  });
  observer.observe(contentDiv, { childList: true, subtree: true });
  contentDiv.querySelectorAll('.markdown').forEach(renderMarkdown);
```

In [ ]:
%%ai 0
I've written a simple HTML generation library -- here's a sample; can you see the basic idea of how to use it?

```python
async def get_list(done: bool|None = None) -> str:
    todos = TODO_LIST if done is None else [
        item for item in TODO_LIST if item.done == done]
    tlist = Ul(Li(item.tag()) for item in todos)
    elems = [H1('TODO list'), tlist, Div(id='current-todo')]
    htmxscr = Script(src="https://unpkg.com/htmx.org@1.9.12",
                     crossorigin="anonymous")
    head = [Title('TODO list'), htmxscr]
    res = Html(Head(*head), Body(*elems))
    return to_xml(res)
```

Yes, I can see the basic idea of how to use your HTML generation library. Here's a breakdown:

- You have defined functions that represent the usual HTML elements, like `Ul`, `Li`, `Div`, `Script`, `Head`, `Body`, and `Html`
- These functions take the element's content as positional arguments, which can be strings or other elements
- Attributes can be specified as keyword arguments, like `Div(id='current-todo')`
- The `Html` element represents the entire document, taking the `Head` and `Body` as arguments
- The `to_xml` function to renders the HTML element tree into an HTML string.

In [ ]:
cell = nb.cells[1]
cell

```json
{ 'cell_type': 'code',
  'execution_count': None,
  'idx_': 1,
  'metadata': {},
  'outputs': [ { 'data': {'text/plain': ['2']},
                 'execution_count': None,
                 'metadata': {},
                 'output_type': 'execute_result'}],
  'source': '# Do some arithmetic\n1+1'}
```

In [ ]:
%%aip 0
Use that to write a python function that converts a code cell (such as the one in `nb` above) into a sensible HTML representation, such as a div with class `cell` containing `input` and `output` divs.

Create a div for each output, give each div the same class as the output type, and for `execute_result` type outputs show markdown (with appropriate class) if it exists, or text/plain (in Pre) otherwise.

In [ ]:
def _out_pre(o): return Pre(strip_ansi_codes('\n'.join(o)).strip())

def out_data(o):
    if o['output_type'] in ('execute_result', 'display_data'):
        if 'text/markdown' in o['data']:
            return Div('\n'.join(o['data']['text/markdown']), _class='markdown')
        return _out_pre(o['data']['text/plain'])
    return _out_pre(o['text'])

def cell_code(cell):
    return Div(
        Pre(Code(cell.source, _class="language-python"), _class='input'),
        *[Div(out_data(o), _class=o['output_type'] + " output") for o in cell.outputs],
        _class='cell cell-code'
    )

In [ ]:
to_xml(cell_code(cell), True)

```xml
<div class="cell cell-code">
  <pre class="input">
    <code class="language-python"># Do some arithmetic
1+1</code>
  </pre>
  <div class="execute_result output">
    <pre>2</pre>
  </div>
</div>
```

In [ ]:
cell = nb.cells[0]
cell

```json
{ 'cell_type': 'markdown',
  'idx_': 0,
  'metadata': {},
  'source': '## A minimal notebook'}
```

In [ ]:
%%aip 0
Write `cell_md` now that outputs a div with the 'markdown' class, using the above example input.

In [ ]:
def cell_md(cell): return Div(cell.source, _class='cell cell-md markdown')

In [ ]:
to_xml(cell_md(cell), True)

```xml
<div class="cell cell-md markdown">## A minimal notebook</div>
```

In [ ]:
%%aip 0
Great. Put it altogether now, taking `nb` as input, and generating a complete HTML page. Don't worry about including the markdown script we discussed -- I'll copy that in myself.

In [ ]:
nb

```json
{ 'cells': [ { 'cell_type': 'markdown',
               'idx_': 0,
               'metadata': {},
               'source': '## A minimal notebook'},
             { 'cell_type': 'code',
               'execution_count': None,
               'idx_': 1,
               'metadata': {},
               'outputs': [ { 'data': {'text/plain': ['2']},
                              'execution_count': None,
                              'metadata': {},
                              'output_type': 'execute_result'}],
               'source': '# Do some arithmetic\n1+1'}],
  'metadata': { 'kernelspec': { 'display_name': 'Python 3 (ipykernel)',
                                'language': 'python',
                                'name': 'python3'}},
  'nbformat': 4,
  'nbformat_minor': 4,
  'path_': '/Users/jhoward/git/nbdev/tests/minimal.ipynb'}
```

In [ ]:
def nb_html(nb, nm='Notebook'):
    head = [Title(nb.get('name', nm))]
    body = [(cell_code,cell_md)[cell.cell_type=='markdown'](cell)
            for cell in nb.cells if cell.cell_type in ('markdown','code')]
    return Html(Head(*head), Body(*body))

In [ ]:
to_xml(nb_html(nb), True)

```xml
<html>
  <head>
    <title>Notebook</title>
  </head>
  <body>
    <div class="cell cell-md markdown">## A minimal notebook</div>
    <div class="cell cell-code">
      <pre class="input">
        <code class="language-python"># Do some arithmetic
1+1</code>
      </pre>
      <div class="execute_result output">
        <pre>2</pre>
      </div>
    </div>
  </body>
</html>
```

In [ ]:
fname = Path.home()/'git/nbdev/tests/docs_test.ipynb'
nb = nbio.read_nb(fname)

In [ ]:
# to_xml(nb_html(nb), True)

### TODO

- Display piccies

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()